In [2]:
import os
# try:
#     os.chdir('workspace')

from options import Options
opt = Options()

In [3]:
import pandas as pd
df = pd.read_json('original/snli_1.0_test.jsonl', lines=True)
df = df.sample(5, random_state=42).reset_index(drop=True)
train_df = df
df

,annotator_labels,captionID,gold_label,pairID,sentence1,sentence1_binary_parse,sentence1_parse,sentence2,sentence2_binary_parse,sentence2_parse
0,"[neutral, neutral, neutral, neutral, neutral]",4609020271.jpg#3,neutral,4609020271.jpg#3r1n,A kid bored in a train with brown hair and his...,( ( A kid ) ( ( bored ( in ( ( ( ( a train ) (...,(ROOT (S (NP (DT A) (NN kid)) (VP (VBD bored) ...,A child is riding the train from New York to B...,( ( A child ) ( ( is ( ( riding ( the train ) ...,(ROOT (S (NP (DT A) (NN child)) (VP (VBZ is) (...
1,"[contradiction, contradiction, contradiction, ...",2313230479.jpg#2,contradiction,2313230479.jpg#2r1c,A brown dog and black and white dog run along ...,( ( ( ( A ( brown dog ) ) and ) ( ( ( black an...,(ROOT (S (NP (NP (DT A) (JJ brown) (NN dog)) (...,Two squirrels run after acorns in the grass.,( ( Two squirrels ) ( ( run ( after ( acorns (...,(ROOT (S (NP (CD Two) (NNS squirrels)) (VP (VB...
2,"[neutral, contradiction, contradiction, neutra...",3729748378.jpg#1,neutral,3729748378.jpg#1r1n,A person in full astronaut suit and gear train...,( ( ( A person ) ( in ( ( full ( astronaut ( s...,(ROOT (S (NP (NP (DT A) (NN person)) (PP (IN i...,A female astronaut adjusting to the feeling of...,( ( ( A ( female astronaut ) ) ( ( ( adjusting...,(ROOT (NP (NP (DT A) (JJ female) (NN astronaut...
3,"[entailment, entailment, contradiction, contra...",4708658738.jpg#0,contradiction,4708658738.jpg#0r1e,An old Indian man dressed in rags sleeps on th...,( ( ( An ( old ( Indian man ) ) ) ( dressed ( ...,(ROOT (S (NP (NP (DT An) (JJ old) (JJ Indian) ...,An old Indian man is dressed up,( ( An ( old ( Indian man ) ) ) ( is ( dressed...,(ROOT (S (NP (DT An) (JJ old) (JJ Indian) (NN ...
4,"[neutral, neutral, neutral, neutral, neutral]",2372820502.jpg#0,neutral,2372820502.jpg#0r1n,A bunch of people are standing all together in...,( ( ( A bunch ) ( of people ) ) ( ( are ( ( st...,(ROOT (S (NP (NP (DT A) (NN bunch)) (PP (IN of...,A group of people are planning something.,( ( ( A group ) ( of people ) ) ( ( are ( plan...,(ROOT (S (NP (NP (DT A) (NN group)) (PP (IN of...


# Build Datasets

In [103]:
import torch
import torch.nn as nn
from utils import *

## Tad bit weird so just change train.py
opt.model = "LSTM"
opt.model_folder = "models/LSTM_TEST"
opt.data_folder = "data/LSTM_TEST"
os.makedirs(opt.model_folder, exist_ok=True)
os.makedirs(opt.data_folder, exist_ok=True)

opt.embedding_dim = 6
opt.hidden_dim = 4

vocab = create_vocab(train_df, opt)
vocab_size = len(vocab)
vocab

{'<unk>': 0,
 '<pad>': 1,
 'a': 2,
 'kid': 3,
 'bored': 4,
 'in': 5,
 'train': 6,
 'with': 7,
 'brown': 8,
 'hair': 9,
 'and': 10,
 'his': 11,
 'head': 12,
 'lying': 13,
 'down': 14,
 'dog': 15,
 'black': 16,
 'white': 17,
 'run': 18,
 'along': 19,
 'the': 20,
 'green': 21,
 'grass': 22,
 'person': 23,
 'full': 24,
 'astronaut': 25,
 'suit': 26,
 'gear': 27,
 'trains': 28,
 'an': 29,
 'underwater': 30,
 'lab': 31,
 'as': 32,
 'scuba': 33,
 'diver': 34,
 'assists': 35,
 'old': 36,
 'indian': 37,
 'man': 38,
 'dressed': 39,
 'rags': 40,
 'sleeps': 41,
 'on': 42,
 'ground': 43,
 'using': 44,
 'backpack': 45,
 'pillow': 46,
 'bunch': 47,
 'of': 48,
 'people': 49,
 'are': 50,
 'standing': 51,
 'all': 52,
 'together': 53,
 'street': 54,
 'building': 55,
 'background': 56,
 'child': 57,
 'is': 58,
 'riding': 59,
 'from': 60,
 'new': 61,
 'york': 62,
 'to': 63,
 'boston': 64,
 'two': 65,
 'squirrels': 66,
 'after': 67,
 'acorns': 68,
 'female': 69,
 'adjusting': 70,
 'feeling': 71,
 'low': 72,

In [104]:
opt.save_options(opt.model_folder)

word_embeddings = nn.Embedding(vocab_size, opt.embedding_dim)

# Replace this line with actual fasttext embeddings
pretrained_fasttext_embeddings  = torch.rand((vocab_size, opt.embedding_dim))

word_embeddings.weight.data.copy_(pretrained_fasttext_embeddings)
torch.save(word_embeddings.state_dict(), f'{opt.data_folder}/word_embeddings.pth')


## Init
vocab = load_vocab(opt)
word_embeddings = nn.Embedding(opt.vocab_size, opt.embedding_dim)
word_embeddings.load_state_dict(torch.load(f'{opt.data_folder}/word_embeddings.pth'))


# Indexing
idx = 0
sentence1, sentence2 = df.loc[idx, ['sentence1', 'sentence2']]
sentence1, sentence2 = clean_text(sentence1), clean_text(sentence2)

y = torch.tensor(le(df.loc[idx, 'gold_label']))

input_ids1 = torch.ones(100, dtype=torch.long)
input_ids2 = torch.ones(100, dtype=torch.long)

for i, word in enumerate(sentence1.split()):
    input_ids1[i] = vocab[word] if word in vocab else 0

for i, word in enumerate(sentence2.split()):
    input_ids2[i] = vocab[word] if word in vocab else 0

embeds1, embeds2 = word_embeddings(input_ids1), word_embeddings(input_ids2)
X = [embeds1, embeds2]

In [7]:
import torch
import torch.nn as nn
from utils import *
from transformers import RobertaTokenizer

## Init
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
sentence1 = df['sentence1'].tolist()
sentence2 = df['sentence2'].tolist()
info = tokenizer(sentence1, sentence2, padding='max_length', max_length=200, return_tensors='pt')

input_ids = info['input_ids']
attention_mask = info['attention_mask']

y = torch.tensor(df['gold_label'].apply(lambda label: le(label)))

## Get idx
idx = 0
input_ids = input_ids[idx]
attention_mask = attention_mask[idx]

X = [input_ids, attention_mask]
y = y[idx]

In [92]:
import torch
import torch.nn as nn
from utils import *

## Init
tokenizer = torch.hub.load('huggingface/pytorch-transformers', 'tokenizer', 'bert-base-uncased')
sentence1 = df['sentence1'].tolist()
sentence2 = df['sentence2'].tolist()
info = tokenizer(sentence1, sentence2, padding='max_length', max_length=200, return_tensors='pt')

input_ids = info['input_ids']
token_type_ids = info['token_type_ids']
attention_mask = info['attention_mask']

y = torch.tensor(df['gold_label'].apply(lambda label: le(label)))


## Get idx
idx = 0
input_ids = input_ids[idx]
token_type_ids = token_type_ids[idx]
attention_mask = attention_mask[idx]

X = [input_ids, token_type_ids, attention_mask]
y = y[idx]

Using cache found in C:\Users\statix/.cache\torch\hub\huggingface_pytorch-transformers_main
c:\Users\statix\miniconda3\envs\nlp\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Build Models

In [18]:
X = [x.view(1,*x.shape) for x in X]
y = y.view(1)

In [77]:
X = [torch.randn(30,*x.shape) for x in X]


In [84]:
lstm = nn.LSTM(opt.embedding_dim, opt.hidden_dim, batch_first=True, bidirectional=True)
embeds1, embeds2 = X

out1, (h, c) = lstm(embeds1)

In [87]:
# Forward and backward concatenated
out1[:, -1, :].shape

torch.Size([30, 8])

In [86]:
# Forward and backward not concatenated
h.shape

torch.Size([2, 30, 4])

In [105]:
# Take the last vector as sentence representation
out1[:, -1, :]

tensor([[ 0.0512,  0.0485, -0.2428,  0.0412,  0.1431,  0.1300, -0.1976, -0.0655],
        [-0.2403,  0.1184, -0.3530,  0.2882,  0.0499, -0.0162, -0.0347, -0.1560],
        [-0.1178, -0.0512, -0.3399,  0.1470, -0.0422,  0.0048,  0.0779, -0.1458],
        [-0.0479,  0.1407, -0.0342, -0.0346,  0.0566,  0.0123, -0.2221,  0.0120],
        [-0.2458,  0.1486, -0.1870,  0.1065,  0.0017, -0.1152, -0.2017, -0.2473],
        [-0.1424, -0.0145, -0.2297, -0.1777,  0.0283,  0.0373, -0.0799, -0.1652],
        [-0.0780,  0.1414, -0.0921, -0.2335,  0.0792,  0.0506,  0.0436, -0.1143],
        [-0.0976, -0.0558,  0.2510, -0.2692,  0.0132, -0.0167,  0.0125, -0.2026],
        [-0.0247,  0.0237, -0.2977,  0.0070,  0.0519,  0.0352, -0.0099, -0.1840],
        [-0.0364,  0.0998, -0.2174,  0.1123,  0.0827,  0.1107, -0.3111,  0.1549],
        [-0.1475,  0.1525, -0.2729,  0.1515,  0.0743, -0.0374, -0.1184, -0.1717],
        [-0.0825,  0.5651, -0.2147,  0.0653,  0.0875, -0.0460,  0.0120, -0.2157],
        [ 0.0415

In [32]:
import torch
import torch.nn as nn
from torch.functional import F


bert = torch.hub.load('huggingface/pytorch-transformers', 'model', 'bert-base-uncased')
linear = nn.Linear(768, 3)

input_ids, token_type_ids, attention_mask = X

out = bert(input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask)

out = out[1] # CLS representation of both sentences
out = linear(out)  # Cross-Entropy Loss takes in the unnormalized logits
out

Using cache found in C:\Users\statix/.cache\torch\hub\huggingface_pytorch-transformers_main


tensor([[ 0.1687,  1.4565, -0.5958]], grad_fn=<AddmmBackward0>)